In [27]:
import warnings
warnings.simplefilter("ignore", UserWarning)

from __future__ import print_function
from pytorch_lightning.metrics.functional import accuracy

from PIL import Image


import os
import random
import numpy as np
import shutil
import copy

import numpy as np
import pandas as pd

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


import torch
import pytorch_lightning as pl
import os

from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

from torch.nn import functional as F
import torchmetrics

from pytorch_lightning.loggers import TensorBoardLogger
from pl_bolts.callbacks import PrintTableMetricsCallback


import math
import torchvision.models as models
from PIL import Image


from skimage import io as iio
from tqdm import tqdm
import cv2

In [2]:
class pre_resnest50(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_resnest50, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size



        backbone = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        
        self.feature_extractor = nn.Sequential(*layers)

        num_target_classes = 800
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)


        return F.log_softmax(x, dim=1)

class pre_resnest50_wo_freeze(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_resnest50_wo_freeze, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.save_hyperparameters()

        resnest50 = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        trained_resnest50 = pre_resnest50.load_from_checkpoint('pre_resnest50_epoch=54-val_loss=6.26.ckpt', learning_rate=0.08, batch_size=128)

        layers_1 = list(resnest50.children())[:-1][0]
        layers_2 = list(resnest50.children())[:-1][1:-2]
        layers_3 = list(resnest50.children())[:-1][-2:]
        self.classifier = list(trained_resnest50.children())[-1]

        self.seq_1 = nn.Sequential(*layers_1)
        self.seq_2 = nn.Sequential(*layers_2)
        self.seq_3 = nn.Sequential(*layers_3)



    def forward(self, x):
        
        x = self.seq_1(x)
        
        self.seq_2.eval()
        with torch.no_grad():
            x = self.seq_2(x)
            
        x = self.seq_3(x).flatten(1)    
        x = self.classifier(x)


        return F.log_softmax(x, dim=1)



In [3]:

class t_brain_right(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(t_brain_right, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.save_hyperparameters()


        tbrain = pre_resnest50_wo_freeze.load_from_checkpoint('resnest50_tbrain_epoch=49-val_loss=0.056.ckpt')

        layers_1 = list(tbrain.children())[1]
        layers_2 = list(tbrain.children())[2]
        layers_3 = list(tbrain.children())[3]
        self.classifier = list(tbrain.children())[0]

        self.seq_1 = nn.Sequential(*layers_1)
        self.seq_2 = nn.Sequential(*layers_2)
        self.seq_3 = nn.Sequential(*layers_3)



    def forward(self, x):
        
        x = self.seq_1(x)
        
        self.seq_2.eval()
        with torch.no_grad():
            x = self.seq_2(x)
            
        x = self.seq_3(x).flatten(1)    
        x = self.classifier(x)


        return F.log_softmax(x, dim=1)


In [4]:

class t_brain_right_eval(pl.LightningModule):
    def __init__(self):
        
        super(t_brain_right_eval, self).__init__()
        


        tbrain = t_brain_right.load_from_checkpoint('resnest50_tbrain_right_epoch=47-val_loss=0.047.ckpt')

        layers_1 = list(tbrain.children())[1:]
        

        self.seq = nn.Sequential(*layers_1)
        self.classifier = list(tbrain.children())[0]


    def forward(self, x):
        
        x = self.seq(x).flatten(1)    
        


        return self.classifier(x)


In [5]:
# index_to_char = {}
# for sub_dict in list(ds.class_to_idx.items()):
#     index_to_char[sub_dict[1]] = sub_dict[0].split('_')[1]
    
# d_file = open('index_to_char.pkl', "wb")
# pickle.dump(index_to_char, d_file)
# d_file.close()

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = t_brain_right_eval()
model.to(device)
model.eval()

Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master


t_brain_right_eval(
  (seq): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): Sequential(
      (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (3): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tr

In [7]:
train_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray_augmented/train'

data_transform = transforms.Compose([
                            transforms.Resize(size = (224, 224)),
                            transforms.Grayscale(num_output_channels=3),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
ds = ImageFolder(train_dir, transform=data_transform)

dl = DataLoader(ds, batch_size=16, shuffle=True)

In [8]:
for i, (x, y) in enumerate(dl):
    if i == 0:
        break

In [9]:
logits = F.log_softmax(model(x.to(device)), dim=1)

In [14]:
torch.cat([y, y])

tensor([120, 102, 514, 210, 288, 118, 184, 175, 596,  88, 126, 572,  27,   1,
        295, 569, 120, 102, 514, 210, 288, 118, 184, 175, 596,  88, 126, 572,
         27,   1, 295, 569])

In [12]:
torch.argmax(logits, dim=1)

tensor([120, 102, 514, 210, 288, 118, 184, 175, 596,  88, 126, 572,  27,   1,
        295, 569], device='cuda:0')

In [10]:
logits

tensor([[-192.8818, -177.5990, -162.1438,  ..., -173.5700, -139.0707,
         -161.0822],
        [-123.1560, -386.8691, -126.1820,  ..., -225.8926, -184.6544,
         -159.4320],
        [-165.6538, -101.2687,  -93.6807,  ..., -160.4950, -124.1876,
          -82.8767],
        ...,
        [-159.1104,    0.0000, -139.7433,  ..., -184.6569, -160.2488,
         -171.6277],
        [ -92.7916, -164.5178,  -82.4240,  ...,  -92.7906,  -58.5318,
          -98.6706],
        [-209.5279, -246.9774, -170.9366,  ..., -119.6509, -117.2481,
         -259.6347]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)

In [13]:
y.shape

torch.Size([64])

In [12]:
x.shape

torch.Size([64, 3, 224, 224])

In [ ]:
for i

In [15]:
# img_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray'
# result_dir = 'C:/Users/bbb50/Desktop/Python/OCR/result'
# result_stat_dir = 'C:/Users/bbb50/Desktop/Python/OCR/result_stat'
# a_file = open("index_to_char.pkl", "rb")
# index_to_char = pickle.load(a_file)

In [3]:
os.listdir(img_dir)

[]

In [2]:
cat_dir = 'C:/Users/bbb50/Desktop/Python/OCR/result'
new_dir = 'C:/Users/bbb50/Desktop/Python/t_brain_new_gray_augmented/t_brain_new_gray_augmented/test'
for i in os.listdir(cat_dir):
    os.mkdir(os.path.join(new_dir, i))

In [11]:
cat_dir = 'C:/Users/bbb50/Desktop/Python/OCR/result'
new_dir = 'C:/Users/bbb50/Desktop/Python/OCR/day1_cat'
for i in os.listdir(cat_dir):
    os.mkdir(os.path.join(new_dir, i))
    
char_list = list(map(lambda x: x.split('_')[1], os.listdir(cat_dir)))

In [ ]:
import pickle

In [15]:
cat_dir = 'C:/Users/bbb50/Desktop/Python/OCR/day1_cat_org'
dir_list = os.listdir(cat_dir)

In [18]:
char_list = list(map(lambda x: x.split('_')[1], os.listdir(new_dir)))

In [17]:
img_dir = 'C:/Users/bbb50/Desktop/Python/OCR/wrong_day1'
new_dir = 'C:/Users/bbb50/Desktop/Python/OCR/day1_cat_org'

In [24]:
for img in tqdm(os.listdir(img_dir)):
    char = img.split('_')[1].split('.')[0]
    src = os.path.join(img_dir, img)
    if char in char_list:
        dst = os.path.join(new_dir, dir_list[char_list.index(char)], img)
        os.rename(src, dst)
    
    

100%|██████████████████████████████████████████████████████████████████████████████| 966/966 [00:00<00:00, 3219.97it/s]


In [25]:
src_dir = 'C:/Users/bbb50/Desktop/Python/OCR/day1_cat_org'
dst_dir = 'C:/Users/bbb50/Desktop/Python/OCR/day1_cat_org_gray'

In [28]:
for char in tqdm(os.listdir(src_dir)):
    sub_dir = os.path.join(src_dir, char)
    for img in os.listdir(sub_dir):
        img_abs = os.path.join(sub_dir, img)
        image_g = cv2.imdecode(np.fromfile(file=img_abs, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
        image_g = cv2.resize(image_g, (224,224), interpolation=cv2.INTER_CUBIC)
        ret3,th3 = cv2.threshold(image_g,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        img_path_d = os.path.join(dst_dir, char, img)
        cv2.imencode('.png', th3)[1].tofile(img_path_d)    
    

100%|███████████████████████████████████████████████████████████████████████████████| 800/800 [00:05<00:00, 134.24it/s]


In [ ]:
image_g = cv2.imdecode(np.fromfile(file=image_name_abs, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
image_g = cv2.resize(image_g, (224,224), interpolation=cv2.INTER_CUBIC)
ret3,th3 = cv2.threshold(image_g,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                
img_path_d = os.path.join(n_sub_path, str(count)+'.png')
cv2.imencode('.png', img)[1].tofile(output_filename)

In [19]:
img_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray_augmented/test'
result_dir = 'C:/Users/bbb50/Desktop/Python/OCR/test_result'
result_stat_dir = 'C:/Users/bbb50/Desktop/Python/OCR/test_result_stat'
a_file = open("index_to_char.pkl", "rb")
index_to_char = pickle.load(a_file)

In [20]:
# for i in os.listdir(img_dir):
#     os.mkdir(os.path.join('C:/Users/bbb50/Desktop/Python/OCR/result', i))

In [21]:
data_transform = transforms.Compose([
                            transforms.Resize(size = (224, 224)),
                            transforms.Grayscale(num_output_channels=3),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [26]:
count_dict = {}
for i in os.listdir('C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray'):
    sub_d = os.path.join('C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray', i)
    count_dict[i] = len(os.listdir(sub_d))

In [41]:
for i, item in enumerate(wrong_dict.items()):
    if len(item[1]) > 1:
        print('{} have {} samples, wrong sample count {}'.format(item[0], count_dict[item[0]], len(item[1])))

108_慶 have 90 samples, wrong sample count 2
112_源 have 96 samples, wrong sample count 2
120_珮 have 72 samples, wrong sample count 2
122_司 have 89 samples, wrong sample count 2
150_農 have 87 samples, wrong sample count 2
165_施 have 97 samples, wrong sample count 2
180_義 have 92 samples, wrong sample count 2
181_紅 have 54 samples, wrong sample count 3
182_翊 have 89 samples, wrong sample count 2
191_馮 have 58 samples, wrong sample count 2
193_樹 have 82 samples, wrong sample count 2
204_戶 have 82 samples, wrong sample count 2
205_黃 have 96 samples, wrong sample count 2
207_于 have 62 samples, wrong sample count 2
209_妹 have 37 samples, wrong sample count 2
212_復 have 27 samples, wrong sample count 2
222_音 have 55 samples, wrong sample count 3
23_限 have 84 samples, wrong sample count 2
254_屬 have 89 samples, wrong sample count 2
263_忠 have 94 samples, wrong sample count 2
26_鑽 have 20 samples, wrong sample count 2
270_具 have 88 samples, wrong sample count 2
28_林 have 94 samples, wrong sample

In [39]:
count_dict[item[0]]

96

['3.png']

In [29]:
for i in wrong_dict.items()

dict_items([('0_宋', ['3.png']), ('100_五', []), ('101_禎', []), ('102_敬', []), ('103_報', []), ('104_柯', []), ('105_婉', []), ('106_清', []), ('107_翰', ['2.png']), ('108_慶', ['2.png', '3.png']), ('109_振', []), ('10_歐', []), ('110_劉', []), ('111_先', []), ('112_源', ['1.png', '2.png']), ('113_佩', []), ('114_生', []), ('115_貞', []), ('116_文', []), ('117_羽', []), ('118_樂', ['0.png']), ('119_大', []), ('11_勤', ['4.png']), ('120_珮', ['1.png', '2.png']), ('121_苑', []), ('122_司', ['0.png', '1.png']), ('123_宸', []), ('124_進', ['4.png']), ('125_慧', []), ('126_視', []), ('127_紋', []), ('128_皇', ['2.png']), ('129_貴', []), ('12_聖', []), ('130_秦', []), ('131_良', []), ('132_冠', []), ('133_鎮', []), ('134_製', ['1.png']), ('135_時', ['4.png']), ('136_位', ['0.png']), ('137_宜', []), ('138_賜', []), ('139_盟', ['4.png']), ('13_築', ['1.png']), ('140_卿', []), ('141_嘉', []), ('142_銓', []), ('143_爾', []), ('144_福', ['3.png']), ('145_峰', []), ('146_寧', []), ('147_雲', []), ('148_百', []), ('149_事', ['3.png']), ('14_代', ['1.p

In [25]:
wrong_dict = {}
acc_dict = {}
no = 0
for char in os.listdir(img_dir):
    
    label = char.split('_')[1]
    sub_dir = os.path.join(img_dir, char)
    
    n = 0
    wrong = 0
    correct_count = 0
    wrong_image = []
    img_num = len(os.listdir(sub_dir))
    
    for img in os.listdir(sub_dir):
        
        img_abs = os.path.join(sub_dir, img)
        img_ = Image.open(img_abs).convert('RGB')
        img_input = data_transform(img_).to(device)
        img_input = img_input.reshape(1,*img_input.shape)
        
        fea = model.seq(img_input)
        logits = F.log_softmax(model(img_input), dim=1)
        
        if index_to_char[torch.argmax(logits, dim=1)[0].item()] == label:
            correct_count += 1
        else:
            wrong_image += [img]
#             if wrong == 0:
#                 wrong_logit_all = csr_matrix(logits.data.cpu().numpy())
# #                 wrong_logit_all = wrong_logit_all_.reshape(1, wrong_logit_all_.shape[0], wrong_logit_all_.shape[1])
#             else:
# #                 wrong_logit_ = logits.data.cpu().numpy()
# #                 wrong_logit = wrong_logit_.reshape(1, wrong_logit_.shape[0], wrong_logit_.shape[1])
# #                 wrong_logit_all = np.stack((wrong_logit_all, wrong_logit), axis=0)
#                 wrong_logit_all = vstack([wrong_logit_all, csr_matrix(logits.data.cpu().numpy())])
                
            wrong += 1
            
        
#         if n == 0:
#             fea_all = csr_matrix(fea.data.cpu().numpy())
#         else:
#             fea_all = vstack([fea_all, csr_matrix(fea.data.cpu().numpy())])
        
        n += 1

#     fea_map = fea_all.toarray().mean(axis=0)
    
#     if no == 0:
#         fea_map_all = fea_map
#     else:
#         fea_map_all = np.stack((fea_map_all, fea_map))

    result_sub = os.path.join(result_dir, char)


#     if wrong != 0:
#         wrong_logit_file = os.path.join(result_sub, 'wrong_logit.npy')
#         np.save(wrong_logit_file, wrong_logit_all.toarray())    
    
    print('label:', label)
    print('accuracy:', correct_count/img_num)
    print('wrong_count:', wrong)
    print('========================================')
    acc_dict[char] =  correct_count/img_num
    wrong_dict[char] = wrong_image
    no += 1
    


label: 宋
accuracy: 0.8
wrong_count: 1
label: 五
accuracy: 1.0
wrong_count: 0
label: 禎
accuracy: 1.0
wrong_count: 0
label: 敬
accuracy: 1.0
wrong_count: 0
label: 報
accuracy: 1.0
wrong_count: 0
label: 柯
accuracy: 1.0
wrong_count: 0
label: 婉
accuracy: 1.0
wrong_count: 0
label: 清
accuracy: 1.0
wrong_count: 0
label: 翰
accuracy: 0.8
wrong_count: 1
label: 慶
accuracy: 0.6
wrong_count: 2
label: 振
accuracy: 1.0
wrong_count: 0
label: 歐
accuracy: 1.0
wrong_count: 0
label: 劉
accuracy: 1.0
wrong_count: 0
label: 先
accuracy: 1.0
wrong_count: 0
label: 源
accuracy: 0.6
wrong_count: 2
label: 佩
accuracy: 1.0
wrong_count: 0
label: 生
accuracy: 1.0
wrong_count: 0
label: 貞
accuracy: 1.0
wrong_count: 0
label: 文
accuracy: 1.0
wrong_count: 0
label: 羽
accuracy: 1.0
wrong_count: 0
label: 樂
accuracy: 0.8
wrong_count: 1
label: 大
accuracy: 1.0
wrong_count: 0
label: 勤
accuracy: 0.8
wrong_count: 1
label: 珮
accuracy: 0.6
wrong_count: 2
label: 苑
accuracy: 1.0
wrong_count: 0
label: 司
accuracy: 0.6
wrong_count: 2
label: 宸
acc

label: 永
accuracy: 0.8
wrong_count: 1
label: 社
accuracy: 1.0
wrong_count: 0
label: 順
accuracy: 1.0
wrong_count: 0
label: 珊
accuracy: 0.8
wrong_count: 1
label: 曾
accuracy: 1.0
wrong_count: 0
label: 慈
accuracy: 1.0
wrong_count: 0
label: 連
accuracy: 1.0
wrong_count: 0
label: 名
accuracy: 0.8
wrong_count: 1
label: 謙
accuracy: 1.0
wrong_count: 0
label: 憶
accuracy: 1.0
wrong_count: 0
label: 帝
accuracy: 1.0
wrong_count: 0
label: 紘
accuracy: 1.0
wrong_count: 0
label: 戶
accuracy: 0.6
wrong_count: 2
label: 黃
accuracy: 0.6
wrong_count: 2
label: 原
accuracy: 0.8
wrong_count: 1
label: 于
accuracy: 0.6
wrong_count: 2
label: 陞
accuracy: 1.0
wrong_count: 0
label: 妹
accuracy: 0.6
wrong_count: 2
label: 千
accuracy: 1.0
wrong_count: 0
label: 姚
accuracy: 1.0
wrong_count: 0
label: 榮
accuracy: 1.0
wrong_count: 0
label: 復
accuracy: 0.6
wrong_count: 2
label: 坊
accuracy: 0.8
wrong_count: 1
label: 彭
accuracy: 1.0
wrong_count: 0
label: 侯
accuracy: 0.8
wrong_count: 1
label: 會
accuracy: 1.0
wrong_count: 0
label: 勝
acc

label: 本
accuracy: 1.0
wrong_count: 0
label: 賓
accuracy: 0.8
wrong_count: 1
label: 林
accuracy: 0.6
wrong_count: 2
label: 總
accuracy: 0.8
wrong_count: 1
label: 份
accuracy: 0.8
wrong_count: 1
label: 餐
accuracy: 1.0
wrong_count: 0
label: 君
accuracy: 1.0
wrong_count: 0
label: 介
accuracy: 1.0
wrong_count: 0
label: 璟
accuracy: 0.8
wrong_count: 1
label: 策
accuracy: 1.0
wrong_count: 0
label: 韻
accuracy: 1.0
wrong_count: 0
label: 仲
accuracy: 1.0
wrong_count: 0
label: 裝
accuracy: 1.0
wrong_count: 0
label: 潔
accuracy: 1.0
wrong_count: 0
label: 楊
accuracy: 1.0
wrong_count: 0
label: 蕙
accuracy: 1.0
wrong_count: 0
label: 育
accuracy: 0.6
wrong_count: 2
label: 鄭
accuracy: 1.0
wrong_count: 0
label: 祥
accuracy: 0.8
wrong_count: 1
label: 醫
accuracy: 1.0
wrong_count: 0
label: 儀
accuracy: 1.0
wrong_count: 0
label: 森
accuracy: 0.8
wrong_count: 1
label: 蓉
accuracy: 1.0
wrong_count: 0
label: 勁
accuracy: 1.0
wrong_count: 0
label: 亞
accuracy: 1.0
wrong_count: 0
label: 詮
accuracy: 1.0
wrong_count: 0
label: 善
acc

label: 翠
accuracy: 1.0
wrong_count: 0
label: 協
accuracy: 1.0
wrong_count: 0
label: 行
accuracy: 1.0
wrong_count: 0
label: 企
accuracy: 0.8
wrong_count: 1
label: 厚
accuracy: 0.8
wrong_count: 1
label: 吟
accuracy: 1.0
wrong_count: 0
label: 鈞
accuracy: 0.8
wrong_count: 1
label: 鳳
accuracy: 1.0
wrong_count: 0
label: 程
accuracy: 0.6
wrong_count: 2
label: 亮
accuracy: 1.0
wrong_count: 0
label: 王
accuracy: 0.8
wrong_count: 1
label: 媛
accuracy: 1.0
wrong_count: 0
label: 白
accuracy: 0.8
wrong_count: 1
label: 浩
accuracy: 0.8
wrong_count: 1
label: 霖
accuracy: 0.8
wrong_count: 1
label: 思
accuracy: 0.8
wrong_count: 1
label: 球
accuracy: 0.8
wrong_count: 1
label: 貝
accuracy: 0.8
wrong_count: 1
label: 光
accuracy: 1.0
wrong_count: 0
label: 運
accuracy: 1.0
wrong_count: 0
label: 漁
accuracy: 0.8
wrong_count: 1
label: 活
accuracy: 0.6
wrong_count: 2
label: 錦
accuracy: 0.8
wrong_count: 1
label: 菊
accuracy: 1.0
wrong_count: 0
label: 卓
accuracy: 1.0
wrong_count: 0
label: 竑
accuracy: 1.0
wrong_count: 0
label: 塑
acc

label: 蘭
accuracy: 1.0
wrong_count: 0
label: 學
accuracy: 1.0
wrong_count: 0
label: 研
accuracy: 1.0
wrong_count: 0
label: 舜
accuracy: 1.0
wrong_count: 0
label: 廷
accuracy: 1.0
wrong_count: 0
label: 記
accuracy: 1.0
wrong_count: 0
label: 章
accuracy: 0.8
wrong_count: 1
label: 和
accuracy: 1.0
wrong_count: 0
label: 彥
accuracy: 1.0
wrong_count: 0
label: 菱
accuracy: 1.0
wrong_count: 0
label: 融
accuracy: 0.8
wrong_count: 1
label: 雪
accuracy: 0.8
wrong_count: 1
label: 駿
accuracy: 1.0
wrong_count: 0
label: 舒
accuracy: 0.8
wrong_count: 1
label: 巫
accuracy: 1.0
wrong_count: 0
label: 蔡
accuracy: 1.0
wrong_count: 0
label: 鴻
accuracy: 0.8
wrong_count: 1
label: 太
accuracy: 1.0
wrong_count: 0
label: 診
accuracy: 1.0
wrong_count: 0
label: 匯
accuracy: 0.8
wrong_count: 1
label: 律
accuracy: 1.0
wrong_count: 0
label: 博
accuracy: 1.0
wrong_count: 0
label: 碧
accuracy: 1.0
wrong_count: 0
label: 呂
accuracy: 1.0
wrong_count: 0
label: 委
accuracy: 1.0
wrong_count: 0
label: 聰
accuracy: 1.0
wrong_count: 0
label: 理
acc

label: 康
accuracy: 0.8
wrong_count: 1
label: 御
accuracy: 0.8
wrong_count: 1
label: 萬
accuracy: 1.0
wrong_count: 0
label: 菁
accuracy: 1.0
wrong_count: 0
label: 調
accuracy: 0.8
wrong_count: 1
label: 三
accuracy: 1.0
wrong_count: 0
label: 成
accuracy: 1.0
wrong_count: 0
label: 實
accuracy: 0.6
wrong_count: 2
label: 胡
accuracy: 1.0
wrong_count: 0
label: 任
accuracy: 1.0
wrong_count: 0
label: 克
accuracy: 1.0
wrong_count: 0
label: 設
accuracy: 1.0
wrong_count: 0
label: 薛
accuracy: 1.0
wrong_count: 0
label: 威
accuracy: 0.8
wrong_count: 1
label: 戴
accuracy: 1.0
wrong_count: 0
label: 啟
accuracy: 1.0
wrong_count: 0
label: 天
accuracy: 1.0
wrong_count: 0
label: 旻
accuracy: 0.8
wrong_count: 1
label: 易
accuracy: 1.0
wrong_count: 0
label: 邦
accuracy: 0.8
wrong_count: 1
label: 廈
accuracy: 1.0
wrong_count: 0
label: 村
accuracy: 0.8
wrong_count: 1
label: 作
accuracy: 1.0
wrong_count: 0
label: 皓
accuracy: 0.6
wrong_count: 2
label: 酒
accuracy: 0.6
wrong_count: 2
label: 枝
accuracy: 1.0
wrong_count: 0
label: 蓮
acc

label: 松
accuracy: 1.0
wrong_count: 0
label: 創
accuracy: 0.8
wrong_count: 1
label: 服
accuracy: 1.0
wrong_count: 0
label: 車
accuracy: 0.8
wrong_count: 1
label: 料
accuracy: 1.0
wrong_count: 0
label: 綺
accuracy: 1.0
wrong_count: 0
label: 鍾
accuracy: 0.8
wrong_count: 1
label: 桂
accuracy: 0.8
wrong_count: 1
label: 蔣
accuracy: 0.8
wrong_count: 1
label: 張
accuracy: 1.0
wrong_count: 0
label: 秉
accuracy: 1.0
wrong_count: 0
label: 翔
accuracy: 0.8
wrong_count: 1
label: 芬
accuracy: 0.8
wrong_count: 1
label: 資
accuracy: 0.8
wrong_count: 1
label: 腦
accuracy: 1.0
wrong_count: 0
label: 何
accuracy: 0.8
wrong_count: 1
label: 毅
accuracy: 0.8
wrong_count: 1
label: 靜
accuracy: 0.8
wrong_count: 1
label: 常
accuracy: 1.0
wrong_count: 0
label: 能
accuracy: 0.6
wrong_count: 2
label: 油
accuracy: 0.8
wrong_count: 1
label: 所
accuracy: 0.8
wrong_count: 1
label: 淑
accuracy: 0.8
wrong_count: 1
label: 麗
accuracy: 1.0
wrong_count: 0
label: 姜
accuracy: 0.6
wrong_count: 2
label: 泰
accuracy: 1.0
wrong_count: 0
label: 侑
acc

label: 伯
accuracy: 1.0
wrong_count: 0
label: 齊
accuracy: 0.8
wrong_count: 1
label: 賢
accuracy: 1.0
wrong_count: 0
label: 人
accuracy: 1.0
wrong_count: 0
label: 藥
accuracy: 1.0
wrong_count: 0
label: 土
accuracy: 1.0
wrong_count: 0
label: 群
accuracy: 1.0
wrong_count: 0
label: 雅
accuracy: 0.8
wrong_count: 1
label: 守
accuracy: 1.0
wrong_count: 0
label: 鈦
accuracy: 1.0
wrong_count: 0
label: 僑
accuracy: 0.6
wrong_count: 2
label: 莊
accuracy: 0.8
wrong_count: 1
label: 璃
accuracy: 1.0
wrong_count: 0
label: 斌
accuracy: 1.0
wrong_count: 0
label: 豐
accuracy: 1.0
wrong_count: 0
label: 專
accuracy: 1.0
wrong_count: 0
label: 數
accuracy: 0.8
wrong_count: 1
label: 銷
accuracy: 1.0
wrong_count: 0
label: 允
accuracy: 1.0
wrong_count: 0
label: 昆
accuracy: 1.0
wrong_count: 0
label: 妮
accuracy: 1.0
wrong_count: 0
label: 首
accuracy: 0.8
wrong_count: 1
label: 籌
accuracy: 0.4
wrong_count: 3
label: 仁
accuracy: 0.8
wrong_count: 1
label: 迅
accuracy: 0.8
wrong_count: 1
label: 茶
accuracy: 0.8
wrong_count: 1
label: 影
acc

In [23]:
wrong_dict

{'0_宋': ['3.png'],
 '100_五': [],
 '101_禎': [],
 '102_敬': [],
 '103_報': [],
 '104_柯': [],
 '105_婉': [],
 '106_清': [],
 '107_翰': ['2.png'],
 '108_慶': ['2.png', '3.png'],
 '109_振': [],
 '10_歐': [],
 '110_劉': [],
 '111_先': [],
 '112_源': ['1.png', '2.png'],
 '113_佩': [],
 '114_生': [],
 '115_貞': [],
 '116_文': [],
 '117_羽': [],
 '118_樂': ['0.png'],
 '119_大': [],
 '11_勤': ['4.png'],
 '120_珮': ['1.png', '2.png'],
 '121_苑': [],
 '122_司': ['0.png', '1.png'],
 '123_宸': [],
 '124_進': ['4.png'],
 '125_慧': [],
 '126_視': [],
 '127_紋': [],
 '128_皇': ['2.png'],
 '129_貴': [],
 '12_聖': [],
 '130_秦': [],
 '131_良': [],
 '132_冠': [],
 '133_鎮': [],
 '134_製': ['1.png'],
 '135_時': ['4.png'],
 '136_位': ['0.png'],
 '137_宜': [],
 '138_賜': [],
 '139_盟': ['4.png'],
 '13_築': ['1.png'],
 '140_卿': [],
 '141_嘉': [],
 '142_銓': [],
 '143_爾': [],
 '144_福': ['3.png']}

In [175]:
wrong_dict = {}
acc_dict = {}
no = 0
for char in os.listdir(img_dir)[:50]:
    
    label = char.split('_')[1]
    sub_dir = os.path.join(img_dir, char)
    
    n = 0
    wrong = 0
    correct_count = 0
    wrong_image = []
    img_num = len(os.listdir(sub_dir))
    
    for img in os.listdir(sub_dir):
        
        img_abs = os.path.join(sub_dir, img)
        img_ = Image.open(img_abs).convert('RGB')
        img_input = data_transform(img_).to(device)
        img_input = img_input.reshape(1,*img_input.shape)
        
        fea = model.seq(img_input)
        logits = F.log_softmax(model(img_input), dim=1)
        
        if index_to_char[torch.argmax(logits, dim=1)[0].item()] == label:
            correct_count += 1
        else:
            wrong_image += [img]
            if wrong == 0:
                wrong_logit_all = csr_matrix(logits.data.cpu().numpy())
#                 wrong_logit_all = wrong_logit_all_.reshape(1, wrong_logit_all_.shape[0], wrong_logit_all_.shape[1])
            else:
#                 wrong_logit_ = logits.data.cpu().numpy()
#                 wrong_logit = wrong_logit_.reshape(1, wrong_logit_.shape[0], wrong_logit_.shape[1])
#                 wrong_logit_all = np.stack((wrong_logit_all, wrong_logit), axis=0)
                wrong_logit_all = vstack([wrong_logit_all, csr_matrix(logits.data.cpu().numpy())])
                
            wrong += 1
            
        
        if n == 0:
            fea_all = csr_matrix(fea.data.cpu().numpy())
        else:
            fea_all = vstack([fea_all, csr_matrix(fea.data.cpu().numpy())])
        
        n += 1

    fea_map = fea_all.toarray().mean(axis=0)
    
    if no == 0:
        fea_map_all = fea_map
    else:
        fea_map_all = np.stack((fea_map_all, fea_map))

    result_sub = os.path.join(result_dir, char)


    if wrong != 0:
        wrong_logit_file = os.path.join(result_sub, 'wrong_logit.npy')
        np.save(wrong_logit_file, wrong_logit_all.toarray())    
    
    print('label:', label)
    print('accuracy:', correct_count/img_num)
    print('wrong_count:', wrong)
    print('========================================')
    acc_dict[char] =  correct_count/img_num
    wrong_dict[char] = wrong_image
    no += 1
    


label: 宋
accuracy: 0.9896907216494846
wrong_count: 1
label: 五
accuracy: 1.0
wrong_count: 0


ValueError: all input arrays must have the same shape

In [181]:
fea_map_all

<800x2048 sparse matrix of type '<class 'numpy.float32'>'
	with 46946 stored elements in Compressed Sparse Row format>

In [ ]:
ub_fea_mat = ub_fea_mat.tocsr()
sparse.save_npz('C:/Users/bbb50/Desktop/Python/hw3/Yelp/ub_fea_mat.npz', ub_fea_mat, True) 

In [184]:
acc_dict_file = os.path.join(result_stat_dir, 'acc_dict.pkl')
wrong_dict_file = os.path.join(result_stat_dir, 'wrong_dict.pkl')    
d_file = open(acc_dict_file, "wb")
pickle.dump(acc_dict, d_file)
d_file.close()

d_file = open(wrong_dict_file, "wb")
pickle.dump(wrong_dict, d_file)
d_file.close()

In [191]:
img_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray'
# result_dir = 'C:/Users/bbb50/Desktop/Python/OCR/result'
# result_stat_dir = 'C:/Users/bbb50/Desktop/Python/OCR/result_stat'

In [192]:
dst_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray_wrong'

In [190]:
os.path.join(img_dir, os.listdir(img_dir)[0] , wrong_dict[os.listdir(img_dir)[0]][0])

'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray\\0_宋\\82.png'

In [193]:

for i in tqdm(os.listdir(img_dir)):
    if len(wrong_dict[i]) != 0:
        for img in wrong_dict[i]:
            src = os.path.join(img_dir, i , img)
            dst = os.path.join(dst_dir, i + img)
            os.rename(src, dst)

100%|██████████████████████████████████████████████████████████████████████████████| 800/800 [00:00<00:00, 5928.73it/s]


In [185]:
wrong_dict

{'0_宋': ['82.png'],
 '100_五': [],
 '101_禎': [],
 '102_敬': [],
 '103_報': [],
 '104_柯': [],
 '105_婉': [],
 '106_清': [],
 '107_翰': ['45.png'],
 '108_慶': ['45.png', '60.png'],
 '109_振': [],
 '10_歐': [],
 '110_劉': [],
 '111_先': [],
 '112_源': ['45.png', '60.png'],
 '113_佩': [],
 '114_生': [],
 '115_貞': [],
 '116_文': [],
 '117_羽': ['88.png'],
 '118_樂': ['0.png'],
 '119_大': [],
 '11_勤': ['82.png'],
 '120_珮': ['18.png', '45.png'],
 '121_苑': [],
 '122_司': ['0.png', '18.png'],
 '123_宸': [],
 '124_進': ['82.png'],
 '125_慧': [],
 '126_視': [],
 '127_紋': [],
 '128_皇': ['45.png'],
 '129_貴': [],
 '12_聖': [],
 '130_秦': [],
 '131_良': [],
 '132_冠': [],
 '133_鎮': [],
 '134_製': ['18.png'],
 '135_時': ['82.png'],
 '136_位': ['0.png'],
 '137_宜': [],
 '138_賜': [],
 '139_盟': ['82.png'],
 '13_築': ['18.png', '20.png'],
 '140_卿': [],
 '141_嘉': [],
 '142_銓': [],
 '143_爾': [],
 '144_福': ['60.png'],
 '145_峰': [],
 '146_寧': [],
 '147_雲': [],
 '148_百': [],
 '149_事': ['60.png'],
 '14_代': ['18.png'],
 '150_農': ['18.png', '45

In [ ]:
wrong_dict = {}
acc_dict = {}
char = os.listdir(img_dir)[0]
    
label = char.split('_')[1]
sub_dir = os.path.join(img_dir, char)

n = 0
wrong = 0
correct_count = 0
wrong_image = []
img_num = len(os.listdir(sub_dir))

for img in os.listdir(sub_dir):

    img_abs = os.path.join(sub_dir, img)
    img_ = Image.open(img_abs).convert('RGB')
    img_input = data_transform(img_).to(device)
    img_input = img_input.reshape(1,*img_input.shape)

    fea = model.seq(img_input)
    logits = F.log_softmax(model(img_input), dim=1)

    if index_to_char[torch.argmax(logits, dim=1)[0].item()] == label:
        correct_count += 1
    else:
        wrong_image = wrong_image.append[img]
        if wrong == 0:
            wrong_logit_all = logits.data.cpu().numpy()
        else:
            wrong_logit_all = np.stack((wrong_logit_all, logits.data.cpu().numpy()))

        wrong += 1


    if n == 0:
        fea_all = csr_matrix(fea.data.cpu().numpy())
    else:
        fea_all = vstack([fea_all, csr_matrix(fea.data.cpu().numpy())])

    n += 1

result_sub = os.path.join(result_dir, char)

fea_file = os.path.join(result_sub, 'fea_mat.npz')
sparse.save_npz(fea_file, fea_all.tocsr(), True) 

if wrong != 0:
    wrong_logit_file = os.path.join(result_sub, 'wrong_logit.npy')
    np.save(wrong_logit_file, wrong_logit_all)

acc_dict[char] =  correct_count/img_num
wrong_dict[char] = wrong_image

    
acc_dict_file = os.path.join(result_stat_dir, 'acc_dict.pkl')
wrong_dict_file = os.path.join(result_stat_dir, 'wrong_dict.pkl')    
d_file = open(acc_dict_file, "wb")
pickle.dump(acc_dict, d_file)
d_file.close()

d_file = open(wrong_dict_file, "wb")
pickle.dump(wrong_dict, d_file)
d_file.close()

In [94]:
wrong_image

[]

In [98]:
correct_count/img_num

1.0

In [109]:
result_sub = os.path.join(result_dir, char)

fea_file = os.path.join(result_sub, 'fea_mat.npz')
sparse.save_npz(fea_file, fea_all.tocsr(), True) 

if wrong != 0:
    wrong_logit_file = os.path.join(result_sub, 'wrong_logit.npy')
    np.save(wrong_logit_file, wrong_logit_all)

acc_dict[char] =  correct_count/img_num
wrong_dict[char] = wrong_image

    
acc_dict_file = os.path.join(result_stat_dir, 'acc_dict.pkl')
wrong_dict_file = os.path.join(result_stat_dir, 'wrong_dict.pkl')    
d_file = open(acc_dict_file, "wb")
pickle.dump(acc_dict, d_file)
d_file.close()

d_file = open(wrong_dict_file, "wb")
pickle.dump(wrong_dict, d_file)
d_file.close()

In [118]:
fea_all.toarray().mean(axis=0).shape

(2048,)

In [96]:
wrong_dict = {}
acc_dict = {}
char = os.listdir(img_dir)[1]
    
label = char.split('_')[1]
sub_dir = os.path.join(img_dir, char)

n = 0
wrong = 0
correct_count = 0
wrong_image = []
img_num = len(os.listdir(sub_dir))

for img in tqdm(os.listdir(sub_dir)):

    img_abs = os.path.join(sub_dir, img)
    img_ = Image.open(img_abs).convert('RGB')
    img_input = data_transform(img_).to(device)
    img_input = img_input.reshape(1,*img_input.shape)

    fea = model.seq(img_input)
    logits = F.log_softmax(model(img_input), dim=1)

    if index_to_char[torch.argmax(logits, dim=1)[0].item()] == label:
        correct_count += 1
    else:
        wrong_image = wrong_image.append[img]
        if wrong == 0:
            wrong_logit_all = logits.data.cpu().numpy()
        else:
            wrong_logit_all = np.stack((wrong_logit_all, logits.data.cpu().numpy()))

        wrong += 1


    if n == 0:
        fea_all = csr_matrix(fea.data.cpu().numpy())
    else:
        fea_all = vstack([fea_all, csr_matrix(fea.data.cpu().numpy())])

    n += 1

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:11<00:00,  8.13it/s]


'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray\\0_宋'

In [84]:
wrong_dict = {}
acc_dict = {}
char = os.listdir(img_dir)[0]
    
label = char.split('_')[1]
sub_dir = os.path.join(img_dir, char)

n = 0
wrong = 0
correct_count = 0
wrong_image = []
img_num = len(os.listdir(sub_dir))

img = os.listdir(sub_dir)[6]

In [103]:
os.listdir(sub_dir)[18]

'25.png'